In [1]:
from pymongo import MongoClient
import folium
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('datlab','companies_seattle')

In [4]:
queryf = list(coll.find())

In [6]:
def getLocation(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [7]:
for office in queryf:
    value = {"$set": {'location':getLocation(office)}}
    coll.update_one(office, value)

In [8]:
for office in queryf:
    value = {"$unset": {"longitude":""}}
    coll.update_one(office, value)

In [9]:
for office in queryf:
    value = {"$unset": {"latitude":""}}
    coll.update_one(office, value)

In [27]:
df = pd.DataFrame(queryf)
df.number_of_employees.replace('',0,inplace=True)
df = df.sort_values(by='number_of_employees', ascending=False)
df

,_id,,name,category_code,number_of_employees,founded_year,description,total_money_raised,latitude,city,location
17,5dd6546985bd26906bbafa5a,6,Decho,web,150.0,2008,,$0,47.602416,Seattle,"{'type': 'Point', 'coordinates': [-122.3326145..."
10,5dd6546985bd26906bbafa53,16,Z2Live,games_video,130.0,2009,Mobile Games,$6.5M,47.610301,Seattle,"{'type': 'Point', 'coordinates': [-122.3399782..."
7,5dd6546985bd26906bbafa50,14,Optify,software,45.0,2008,Digital Marketing Software,$10.8M,47.603364,Seattle,"{'type': 'Point', 'coordinates': [-122.333359,..."
6,5dd6546985bd26906bbafa4f,13,L4 Mobile,mobile,25.0,2008,Concept to delivery- mobile services,$4.8M,47.676762,Seattle,"{'type': 'Point', 'coordinates': [-122.2049192..."
15,5dd6546985bd26906bbafa58,5,findwell,web,7.0,2008,Real Estate Search,$0,47.627368,Seattle,"{'type': 'Point', 'coordinates': [-122.34229, ..."
4,5dd6546985bd26906bbafa4d,9,able2know,web,5.0,2008,Question and Answer Community,$0,47.599091,Seattle,"{'type': 'Point', 'coordinates': [-122.335109,..."
16,5dd6546985bd26906bbafa59,17,GroupieGuide,web,4.0,2008,Free Group Website,$0,47.604279,Seattle,"{'type': 'Point', 'coordinates': [-122.3374372..."
20,5dd6546985bd26906bbafa5d,20,Flaggpole,web,4.0,2009,location-based social network,$0,47.620973,Seattle,"{'type': 'Point', 'coordinates': [-122.347276,..."
2,5dd6546985bd26906bbafa4b,3,Gimigo,analytics,4.0,2013,Click. Change. Done.™,$0,44.859587,Seattle,"{'type': 'Point', 'coordinates': [-93.226503, ..."
9,5dd6546985bd26906bbafa52,15,Twibs,advertising,3.0,2008,Directory of businesses on Twitter,$0,47.718192,Seattle,"{'type': 'Point', 'coordinates': [-122.284674,..."


In [21]:
db, coll1 = connectCollection('datlab','companies_seattle')

In [23]:
queryf1 = list(coll.find())

In [24]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)

In [25]:
for company in queryf1:
    folium.Marker(company['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)

In [26]:
map_city